# Imports

In [48]:
import pandas as pd
import numpy as np
import psycopg2
import psycopg2.extras

from dotenv import load_dotenv
import os
from bs4 import BeautifulSoup
import requests

# Constants

In [31]:
dotenv_path = ("/Users/maukanmir/Documents/Machine-Learning/Web-Scraping-Code/Player-Salaries/dot.env")
load_dotenv(dotenv_path)

DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
TABLE_NAME = "player_stats"

# Functions

In [32]:
def extract_nba_stats(html_content, year):
  
  soup = BeautifulSoup(html_content, 'html.parser')
  master_df = []
  tds = soup.find_all("td")
  player_stats = [td.text.strip() for td in tds]
  player_columns = ["Rank", "Name", "Team", "GP", "MPG", "PPG", "FGM", "FGA", "FG%", "3PM", "3PA", "3P%", "FTM", "FTA", "FT%", "ORB", "DRB", "RPG", "APG", "SPG", "BPG", "TOV", "PF"]

  count = 0
  player_dict = {}
  for idx, player_stat in enumerate(player_stats):
    
    if count == 22:
      master_df.append(player_dict)
      player_dict = {}
      count = 0
    else:
      col = player_columns[count]
      player_dict[col] = player_stat
      count +=1
  
  df = pd.DataFrame(master_df)
  df["season"] = year
  return df

In [33]:
years = [str(year+1) for year in range(1990, 2024)]
pages = range(1, 4)

In [36]:
all_data= pd.DataFrame()
for year in years:
  for page in pages:
    base_url = f"https://basketball.realgm.com/nba/stats/{year}/Averages/Qualified/points/All/desc/{page}/Regular_Season"
    response = requests.get(base_url)
    try:
      df = extract_nba_stats(response.text, year)
      all_data = pd.concat([all_data, df])
    except Exception as error:
      print(f" THe error is {error}")

In [39]:
all_data.drop("Rank", inplace=True, axis=1)

In [41]:
all_data['id'] = range(1, len(all_data) + 1)

In [72]:
try:
    with psycopg2.connect(
        host=DB_HOST,
        database=DB_NAME,
        user=DB_USER,
        password=DB_PASSWORD
    ) as conn:
        with conn.cursor() as cur:
            cur.execute(f"DROP TABLE IF EXISTS {TABLE_NAME};")
            cur.execute(f"""
                CREATE TABLE {TABLE_NAME} (
                    id SERIAL PRIMARY KEY,
                    Name VARCHAR,
                    Team VARCHAR,
                    GP INTEGER,
                    MPG FLOAT,
                    PPG FLOAT,
                    FGM FLOAT,
                    FGA FLOAT,
                    "FG%" FLOAT,
                    "3PM" FLOAT,
                    "3PA" FLOAT,
                    "3P%" FLOAT,
                    FTM FLOAT,
                    FTA FLOAT,
                    "FT%" FLOAT,
                    ORB FLOAT,
                    DRB FLOAT,
                    RPG FLOAT,
                    APG FLOAT,
                    SPG FLOAT,
                    BPG FLOAT,
                    TOV FLOAT
                );
            """)

            insert_query = f'INSERT INTO {TABLE_NAME} (Name, Team, GP, MPG, PPG, FGM, FGA, "FG%", "3PM", "3PA", "3P%", FTM, FTA, "FT%", ORB, DRB, RPG, APG, SPG, BPG, TOV) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'

            # Verify column selection
            selected_columns = ['Name', 'Team', 'GP', 'MPG', 'PPG', 'FGM', 'FGA', 'FG%', '3PM', '3PA', '3P%', 'FTM', 'FTA', 'FT%', 'ORB', 'DRB', 'RPG', 'APG', 'SPG', 'BPG', 'TOV']
            rows_to_insert = all_data[selected_columns].values.tolist()

            # Execute the insertion of data
            cur.executemany(insert_query, rows_to_insert)
            conn.commit()

            print("Data written to the database.")
except Exception as e:
    print(f"Database failed to upload data. The error is: {e}")


Database failed to upload data. The error is: list index out of range


In [ ]:

# # Create Chrome options if needed (optional)
# opts = Options()
# opts.add_argument("--headless")  # Example option, run Chrome in headless mode

# # Initialize the WebDriver using webdriver-manager
# web_driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)

# # Example usage
# web_driver.get("https://www.nba.com/stats/leaders?SeasonType=Regular+Season&Season=2023-24")
# print(web_driver.title)  # Prints the title of the page

In [96]:
all_data[all_data["Name"] == "Michael Jordan"]

,Name,Team,GP,MPG,PPG,FGM,FGA,FG%,3PM,3PA,...,FTA,FT%,ORB,DRB,RPG,APG,SPG,BPG,TOV,season
0,Michael Jordan,CHI,82,37.0,31.5,12.1,22.4,.539,0.4,1.1,...,8.2,.851,1.4,4.6,6.0,5.5,2.7,1.0,2.5,1991
0,Michael Jordan,CHI,80,38.8,30.1,11.8,22.7,.519,0.3,1.3,...,7.4,.832,1.1,5.3,6.4,6.1,2.3,0.9,2.5,1992
0,Michael Jordan,CHI,78,39.3,32.6,12.7,25.7,.495,1.0,2.9,...,7.3,.837,1.7,5.0,6.7,5.5,2.8,0.8,2.7,1993
0,Michael Jordan,CHI,82,37.7,30.4,11.2,22.6,.495,1.4,3.2,...,8.0,.834,1.8,4.8,6.6,4.3,2.2,0.5,2.4,1996
0,Michael Jordan,CHI,82,37.9,29.6,11.2,23.1,.486,1.4,3.6,...,7.0,.833,1.4,4.5,5.9,4.3,1.7,0.5,2.0,1997
0,Michael Jordan,CHI,82,38.8,28.7,10.7,23.1,.465,0.4,1.5,...,8.8,.784,1.6,4.2,5.8,3.5,1.7,0.5,2.3,1998
9,Michael Jordan,WAS,60,34.9,22.9,9.2,22.1,.416,0.2,0.9,...,5.6,.790,0.8,4.8,5.7,5.2,1.4,0.4,2.7,2002
24,Michael Jordan,WAS,82,37.0,20.0,8.3,18.6,.445,0.2,0.7,...,4.0,.821,0.9,5.2,6.1,3.8,1.5,0.5,2.1,2003


In [98]:
try:
    with psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        ) as conn:
        with conn.cursor() as cur:
            cur.execute(f"DROP TABLE IF EXISTS {TABLE_NAME};")
            cur.execute(f"""
                CREATE TABLE {TABLE_NAME} (
                    id SERIAL PRIMARY KEY,
                    Name VARCHAR,
                    Team VARCHAR,
                    GP INTEGER,
                    MPG FLOAT,
                    PPG FLOAT,
                    FGM FLOAT,
                    FGA FLOAT,
                    "FG%" FLOAT,
                    "3PM" FLOAT,
                    "3PA" FLOAT,
                    "3P%" FLOAT,
                    FTM FLOAT,
                    FTA FLOAT,
                    "FT%" FLOAT,
                    ORB FLOAT,
                    DRB FLOAT,
                    RPG FLOAT,
                    APG FLOAT,
                    SPG FLOAT,
                    BPG FLOAT,
                    TOV FLOAT,
                    Season INTEGER
                );
            """)

            # Assuming 'id' is auto-generated and excluding it from the insert
            insert_query = f'''
                INSERT INTO {TABLE_NAME} (
                    Name, Team, GP, MPG, PPG, FGM, FGA, "FG%", "3PM", "3PA", "3P%", FTM, FTA, "FT%", ORB, DRB, RPG, APG, SPG, BPG, TOV, Season
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
            '''

            # Exclude 'id' from the data tuples
            rows_to_insert = [
                (
                    row['Name'], row['Team'], row['GP'], row['MPG'], row['PPG'],
                    row['FGM'], row['FGA'], row['FG%'], row['3PM'], row['3PA'],
                    row['3P%'], row['FTM'], row['FTA'], row['FT%'], row['ORB'],
                    row['DRB'], row['RPG'], row['APG'], row['SPG'], row['BPG'],
                    row['TOV'], row['season']
                )
                for index, row in all_data.iterrows()
            ]

            cur.executemany(insert_query, rows_to_insert)
            conn.commit()

            print("Data written to the database.")
except Exception as e:
    print(f"Database Failed to upload Data. The error is: {e}")


Database Failed to upload Data. The error is: tuple index out of range
